<a href="https://colab.research.google.com/github/Gail529/Mental-health/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd 
import numpy as np
import gensim
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
data=pd.read_csv('/content/tweet_emotion_scores.csv')

In [6]:
data['score']=0
data.head()


,Unnamed: 0,user,location,text,tweets,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,score
0,0,app_stayaway,Mumbai,People can also take advantage of your phone a...,people can also take advantage of your phone a...,2,2,1,1,1,1,1,1,1,1,0
1,1,rykorua,Italia,Christmas is a good time to start practicing g...,christmas is a good time to start practicing g...,3,0,0,3,0,1,3,0,1,1,0
2,2,allevin18,"Minneapolis, MN","""Driving in the Midst of #Depression"" https://...",driving in the midst of depression mindwand...,1,2,0,1,0,1,0,1,0,0,0
3,3,DulleyTopBooks,"North Vancouver, B.C. Canada",@BCGovNews May the Prince of Peace give us ALL...,may the prince of peace give u all peace of ou...,4,0,0,2,0,0,2,0,0,2,0
4,4,sadvibes43ver,NaN,Anyone else just pretending they’re ok?\n#sadt...,anyone else just pretending they re ok sadtw...,0,2,1,0,0,0,0,1,0,0,0


In [7]:
data.loc[(data['Positive'] < data['Negative']) & (data['Sadness'] >= 1) , 'score']= '1'


In [8]:
del data['Unnamed: 0']
del data['user']
del data['location']
del data['text']
data.head()

,tweets,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,score
0,people can also take advantage of your phone a...,2,2,1,1,1,1,1,1,1,1,0
1,christmas is a good time to start practicing g...,3,0,0,3,0,1,3,0,1,1,0
2,driving in the midst of depression mindwand...,1,2,0,1,0,1,0,1,0,0,1
3,may the prince of peace give u all peace of ou...,4,0,0,2,0,0,2,0,0,2,0
4,anyone else just pretending they re ok sadtw...,0,2,1,0,0,0,0,1,0,0,1


In [20]:
tweets=data['tweets'].values.tolist()
len(tweets)

1999

In [32]:
tweet_list=[]
for tweet in tweets:
    tokens =  word_tokenize(tweet)
    stop_words=set(stopwords.words("english")) 
    word=[word for word in tokens if word not in stop_words]
    tweet_list.append(word)

    

In [ ]:
tweet_list

In [ ]:
EMBEDDING_DIM=100
model = gensim.models.Word2Vec(sentences=tweet_list,size=EMBEDDING_DIM,window=5,workers=4,min_count=1)
words = list(model.wv.vocab)
print('Vocabulary size: %d',len(words))
print(words)

In [36]:
model.wv.most_similar('depression')

[('anxiety', 0.9959946870803833),
 ('year', 0.99590665102005),
 ('wa', 0.9957513213157654),
 ('diagnosed', 0.99546217918396),
 ('mentalhealth', 0.9951702952384949),
 ('get', 0.9946869015693665),
 ('like', 0.9945575594902039),
 ('people', 0.9944247007369995),
 ('one', 0.9942953586578369),
 ('im', 0.9942030310630798)]

In [39]:
filename = 'tweets_embedding.txt'
model.wv.save_word2vec_format(filename,binary=False)

In [41]:
import os
embeddings_index = {}
f = open(os.path.join('','tweets_embedding.txt'),encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()    

In [43]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [46]:
tok_obj = Tokenizer()
tok_obj.fit_on_texts(tweet_list)
sequences = tok_obj.texts_to_sequences(tweet_list)

word_index=tok_obj.word_index
print(len(words))

max_length = max([len(s.split()) for s in tweets])

tweet_pad =pad_sequences(sequences,maxlen=max_length)
emotion_score=data['score'].values

print('Shape of tweet Tensor:',tweet_pad.shape)
print('Shape of emotion_score Tensor:',emotion_score.shape)



4768
Shape of tweet Tensor: (1999, 29)
Shape of emotion_score Tensor: (1999,)


In [48]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words,EMBEDDING_DIM))

for word,i in word_index.items():
    if i > num_words:
        continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [49]:
print(num_words)

4769


In [52]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant


model = Sequential()
embedding_layer = Embedding(num_words,
                             EMBEDDING_DIM,
                             embeddings_initializer=Constant(embedding_matrix),
                             input_length=max_length,
                             trainable=False )

model.add(embedding_layer)
model.add(GRU(units=32,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam' , metrics=['accuracy'])


In [53]:
VALIDATION_SPLIT = 0.2
indices = np.arange(tweet_pad.shape[0])
np.random.shuffle(indices)
tweet_pad = tweet_pad[indices]
emotion_score = emotion_score[indices]

num_validation_samples = int(VALIDATION_SPLIT * tweet_pad.shape[0])

x_train_pad = tweet_pad[:-num_validation_samples]
y_train = emotion_score[:-num_validation_samples]
x_test_pad = tweet_pad[-num_validation_samples]
y_test = emotion_score[-num_validation_samples]

In [55]:
model.fit(x_train_pad,y_train, batch_size=128 , epochs=25 , validation_data=(x_test_pad,y_test),verbose=2)

ValueError: ignored